


**Security and Privacy in Machine Learning**


**HW3**


**Full Name:**

**Student Number:**

# **Preface (MUST READ :))**
In this notebook you will firstly train a simple CNN and apply the FGSM attack on it to craft adversarial examples, then you wil launch adversaial training on the same CNN architecture, using your method for crafting adversarial examples. Afterwards you will evaluate the  robustness of the defended CNN against PGD attack and then compare the results of this attack on the defended CNN in different settings.

P.S.1 Reminder that the so-called task might be a little bit time consuming even on CUDA! So make sure to start as soon as you can to meet the excercise deadline properly.

P.S.2 It's suggested to save checkpoints for at least every 10 epochs using Colab GPUs.

P.S.3 Don't forget to add the resnet.py file to the same directory of the current notebook on your PC or on Google Drive (if you want to use the drive)

P.S.4 IMPORTANT: In the answers file you will provide at the end for the questions section, you must present a report including the plots and the required values you are asked for, such as the different accuracy values, the success rates and etc for sections 8, 9, 10, 11, 13, 14, 15, 16. the code of these sections is not enough cause the final report will be checked for the values and plots.

P.S.5 BTW Happy New Year :)

# **Training a ResNet18 model with CIFAR10 Dataset**
Use the cells below to  bulid and train a ResNet18 model on CIFAR10 dataset.

# [1] Initial Setups

In [ ]:
from google.colab import drive
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from typing import Type
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from torch.autograd import Variable

# Please use CUDA :\

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
if device == 'cuda':
  print(torch.cuda.get_device_name(0))

Let's try Drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir ./myCIFAR10

from resnet import *

# [2] CIFAR10 Setup

In [ ]:
batch_size = 128
transform = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/myCIFAR10', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='/content/drive/MyDrive/myCIFAR10', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# [3] Training Setup

In [ ]:
model = ResNet18()
model = model.to(device)
model.train()

learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# Use a scheduler if needed

# [4] Train Phase

In [ ]:
epochs = 100
for epoch in range(epochs):  

    print("epoch " + str(epoch))
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1:    # print every 2000 mini-batches #1999
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    

print('Finished Training')

# [5] Evaluating Total Standard Accuracy
Check your model accuracy with the following piece of code

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Standard Accuracy of ResNet18 model on the 10000 test images: {100 * correct / total} %')

# [6] Saving the trained model (Optional)

In [ ]:
model.eval()
model_name = "resnet18_cifar10_model_std"
CIFAR10_model_PATH = "/content/drive/MyDrive/" + model_name
torch.save(model.state_dict(), CIFAR10_model_PATH)

# **Implementing FGSM**

# [7] Fast Gradient Sign Method
You must complete the required sections with proper code for FGSM attack

In [ ]:
class FGSMAttack():

    def __init__(self, model):
        self.model = model
        self.adversarials = []
        self.originals = []
        self.predictions = []
        self.confidences = []
        self.truths = []



    def untargeted_fgsm(self, image, epsilon, gradient):
      
      # Define untargeted fast gradient sign attack pertubation
      # Firstly perturb the image in the proper direction then clamp it between the right values.
      #############################
      # Your code goes here

      #############################

    def __call__(self, image, label, epsilon):
          
      image = image.to(device)
      label = label.to(device)
      
      true_image = image.clone()    
      
      out = self.model(image)
      _, pred = torch.max(out.image, 1)
      
      if pred.item() == label.item():
        
        image.requires_grad = True

        # Now define a loss(use the negative log likelihood loss),
        # Zero the gradients of the model parameters and do the backpropagation
        # Retrieve the gradient of the loss with respect to the input image
        # Finally use the function you defined earlier to craft the perturbed image
        #############################
        # Your code goes here

        #############################

        out2 = self.model(perturbed_image)

        _, pred2 = torch.max(out2.data, 1)

        confidence = F.softmax(out2, dim=1)[0][pred2].data.cpu().numpy()[0]
        
        prob = confidence
        
        # image = Variable(perturbed_image, requires_grad=True)
        
        if pred2.item() == label.item():

          self.adversarials.append(perturbed_image.squeeze().detach().cpu().numpy())
          self.originals.append(true_image.squeeze().detach().cpu().numpy())
          self.predictions.append(pred2.item())
          self.confidences.append(confidence.item())
          self.truths.append(label.item())

            
        return self.adversarials, self.originals, self.predictions, self.confidences, self.truths

# [8] Evaluating FGSM Accuracy of Standard Trained Model

In [ ]:
# For 3 values of epsilon = 4/255, 8/255, 12/255
# Launch the FGSM attack,
# Use the whole testloader to generate a set of adversarial examples on the CIFAR10 testset
# Then use this new testset for calculating the adversarial accuracy of the standard target model
#############################
# Your code goes here

#############################

# [9] Crafting Adversarial Examples with FGSM

In [ ]:
# Using the epsilon value of 8/255
# For the first 5 adversarial images, 
# Print the true label, the model prediction and the confidence of each prediction for each image
#############################
# Your code goes here
    
#############################

# [10] Plotting


In [ ]:
import matplotlib.pyplot as plt
# Now show the 5 original test images mentioned in the previous cell
# And the FGSM perturbation added to each image (with epsilon = 8/255)
# Then the adversarial example created on the same 5 test images using the mentioned FGSM perturbation
#############################
# Your code goes here

#############################

# [11] **Adversarial Training**

In [ ]:
# Now train a ResNet model with FGSM adversarial examples crafted on CIFAR10 trainset
# Then report the clean accuracy of this adversarially trained network on the clean CIFAR10 testset
# Also report the adversarial accuracy of this adversarially trained network against FGSM with epsilon = 8/255 on the CIFAR10 testset
# Try using a scheduler for the learning rate (if needed)
#############################
# Your code goes here

#############################

# **Implementing PGD**

# [12] Projected Gradient Descent
You must complete the required sections with proper code for PGD attack

In [ ]:
class LinfPGDAttack(object):

    def __init__(self, model, epsilon = 8/255, k = 2, alpha = 2/255):
        self.model = model
        self.epsilon = epsilon
        self.steps = k
        self.alpha = alpha

    def __call__(self, image, label):

      out = self.model(image)
      _, pred = torch.max(out.data, 1)
        
      if pred.item() == label.item():

          perturbed_image = image.clone()

          # Now use the input information of the class to implement a k step PGD attack
          # of size alpha while always staying within epsilon distance from the initial point
          # Use cross entropy loss
          #############################
          # Your code goes here

          ############################# 

              # out2 = self.model(perturbed_image)
              # _, pred = torch.max(out2.data, 1)
              # if pred.item() != label.item():
              #   break

      return perturbed_image

# [13] Evaluating PGD Accuracy of Adversarially Trained Model and Standard Trained Model

In [ ]:
# Now launch the attack with the proper inputs for the whole CIFAR10 testset (with epsilon = 8/255)
# on the adversarially trained model and the standard trained model with 2 different step settings: k = 2 and k = 4
# Then print the PGD Accuracy for the images crafted with these 2 settings on the defended model and the standard model

# By PGD Accuracy we mean check if the adversarially trained model could still understand the true label that must be predicted
# Or will the model still get fooled by the PGD attack again after the FGSM adversarial training?
#############################
# Your code goes here

############################# 

# [14] Plotting

In [ ]:
# Now show 5 original test images from the CIFAR10 testset 
# And the PGD perturbation added to each image (with epsilon = 8/255)
# Then the adversarial example created on the same 5 test images using the mentioned PGD perturbation
#############################
# Your code goes here

#############################

# [15] Noisy Input Accuracy

In [ ]:
# Add Gaussian noise with mean of 0 and variance of 6/255 to the CIFAR10 testset
# Evaluate the accuracy of these noisy images on the standard trained model and the adversarially trained model
# What do you conclude from the results of applying noisy inputs and adversarial example inputs to both of the models?
#############################
# Your code goes here

#############################

# [16] Logit VS Epsilon Test

In [ ]:
# Pick the first image x from the CIFAR10 testset
# Make sure the prediction for this image is done correctly by the standard trained model
# Compute the gradient of the model's loss with respect to the image x
# Now for the epsilon values between -0.5 and 0.5 with step size of 0.01 (101 different values),
# Compute x + \epsilon.sign(\nabla_x J(W, x, y))
# Get the logit value for these 101 different images (by logit we mean the softmax input)
# Then plot the logit value vs the epsilon value diagram
# Now choose a random direction instead of \nabla_x J(W, x, y), from the Uniform basis between -1 and 1 for each dimension
# Plot the logit value vs the epsilon value diagram with respect to the new direction
# What is your conclusion based on the results achieved from the two digrams?
#############################
# Your code goes here

#############################